In [1]:
import unicodecsv
import pandas as pd
#read data from CSV file and make it to lists
def read_csv(filename):
    with open(filename, 'rb') as f:
        reader = unicodecsv.DictReader(f)
    
        return list(reader)
    
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import sklearn

In [2]:
data_all = pd.read_csv('../../../2_data/explored/customer.csv')
customer_all=data_all['CUSTOMER_ID']
df_customer =pd.DataFrame(customer_all)
df_customer[:5]

,CUSTOMER_ID
0,2083
1,2102
2,2223
3,2224
4,2284


In [3]:

order_item = pd.read_csv('../../../2_data/explored/order_item.csv')
order = pd.read_csv('../../../2_data/explored/order.csv')
order_succsess =pd.read_csv('../../../2_data/prepared/order_success.csv')
product = pd.read_csv('../../../2_data/explored/product.csv')



goods = product['MATNR']
code=product['MAKTX']
df_goods =pd.DataFrame(goods)
df_code=pd.DataFrame(code)

df_product = pd.concat([df_goods,df_code], axis=1)
merge_id=pd.merge(order_succsess,order_item, on='ORDER_ID')
merge_product=pd.merge(product,merge_id, on='MATNR')



id_cus = merge_product['CUSTOMER_ID']
q=merge_product['QUANTITY']
matnr =merge_product['MATNR']
product_name = merge_product['MAKTX']



#oreder  success




# sum quantity

In [4]:
df2 = pd.concat([id_cus,matnr,q,product_name], axis=1, join='outer')


In [5]:
sum_df1 = df2.groupby(['CUSTOMER_ID','MATNR','MAKTX'])['QUANTITY'].sum().reset_index()
sum_df1.head()


,CUSTOMER_ID,MATNR,MAKTX,QUANTITY
0,21,25017251,SBDS ภาพแขวน #XC55153/XC,2
1,1042,25014525,SBDS เทียนตกแต่ง #5018N/YK,1
2,2082,19086644,MELONA/ตู้ผ้าWV137/DM01/D05LR/ขาว,1
3,2082,19086651,MELONA/โต๊ะทำงานDK120/ขาว,1
4,2082,19113824,BENTE/ตู้เก็บของC60/โซลิด/เดนิม-เทาดำ,1


# remove duplicate

In [6]:
df_id_remove = pd.DataFrame(sum_df1['CUSTOMER_ID'])

no_dup_id = df_id_remove.drop_duplicates(subset=['CUSTOMER_ID'])
no_dup_id[:5]

,CUSTOMER_ID
0,21
1,1042
2,2082
13,2391
14,2463


# setup index

In [7]:
no_dup_id = no_dup_id.reset_index()
no_dup_id['NEW_ID'] = no_dup_id.index + 1
df_new_id = pd.DataFrame(no_dup_id['NEW_ID']) 
df_new_id[:5]

,NEW_ID
0,1
1,2
2,3
3,4
4,5


In [8]:
add_new_id = pd.concat([no_dup_id], axis=1, join='outer')

In [9]:
add_new_id[:5]
del add_new_id['index']


In [10]:
add_new_id[:5]

,CUSTOMER_ID,NEW_ID
0,21,1
1,1042,2
2,2082,3
3,2391,4
4,2463,5


In [11]:
merge_sum_quan = pd.merge(add_new_id,sum_df1,how = 'left' , left_on =['CUSTOMER_ID'], right_on = ['CUSTOMER_ID'])
merge_sum_quan[:5]


,CUSTOMER_ID,NEW_ID,MATNR,MAKTX,QUANTITY
0,21,1,25017251,SBDS ภาพแขวน #XC55153/XC,2
1,1042,2,25014525,SBDS เทียนตกแต่ง #5018N/YK,1
2,2082,3,19086644,MELONA/ตู้ผ้าWV137/DM01/D05LR/ขาว,1
3,2082,3,19086651,MELONA/โต๊ะทำงานDK120/ขาว,1
4,2082,3,19113824,BENTE/ตู้เก็บของC60/โซลิด/เดนิม-เทาดำ,1


# new table

In [12]:
customer_id = merge_sum_quan['NEW_ID']
customer_id_old =merge_sum_quan['CUSTOMER_ID']
matnr1 =merge_sum_quan['MATNR']
quan = merge_sum_quan['QUANTITY']
sum_df2 = pd.concat([customer_id_old,customer_id,matnr1,quan], axis=1, join='outer')
sum_df2[:5]


,CUSTOMER_ID,NEW_ID,MATNR,QUANTITY
0,21,1,25017251,2
1,1042,2,25014525,1
2,2082,3,19086644,1
3,2082,3,19086651,1
4,2082,3,19113824,1


# change matnr to string

In [13]:
str_matnr =sum_df2['MATNR'].apply(str)

In [14]:
sum_df = pd.concat([customer_id_old,customer_id,str_matnr,quan], axis=1, join='outer')

In [15]:
sum_df[:5]

,CUSTOMER_ID,NEW_ID,MATNR,QUANTITY
0,21,1,25017251,2
1,1042,2,25014525,1
2,2082,3,19086644,1
3,2082,3,19086651,1
4,2082,3,19113824,1


In [16]:
sum_df['CUSTOMER_ID'] = sum_df['CUSTOMER_ID'].astype(int).astype(str)


In [17]:
sum_df.dtypes

CUSTOMER_ID    object
NEW_ID          int64
MATNR          object
QUANTITY        int64
dtype: object

In [18]:
np.unique(sum_df['QUANTITY'])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 13, 18, 24, 39])

# dataframe of goods

In [19]:
matnr2 = df2['MATNR']
maktx= df2['MAKTX']
df_goods = pd.concat([matnr2,maktx], axis=1, join='outer')
df_goods[:5]

,MATNR,MAKTX
0,19047936,PATISIA/ตู้ข้างเตียงNT45/ขาว
1,19047936,PATISIA/ตู้ข้างเตียงNT45/ขาว
2,19047936,PATISIA/ตู้ข้างเตียงNT45/ขาว
3,19047936,PATISIA/ตู้ข้างเตียงNT45/ขาว
4,19047936,PATISIA/ตู้ข้างเตียงNT45/ขาว


# pivot table 
(โดยให้ customer_id เป็นแถว และรหัสของสินค้าเป็น columns)

In [20]:
pivot = sum_df.pivot_table(index='MATNR', columns='CUSTOMER_ID', values='QUANTITY', aggfunc=np.sum, fill_value = 0)
pivot.index.name = None
pivot.columns.name = None
pivot

,100085,100522,100622,100683,100684,100806,100822,100843,100862,100982,...,99087,99222,99322,99582,99702,99782,99885,99887,99923,99946
19000158,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19000346,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19000900,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19005144,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19008811,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19008833,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19009152,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19009245,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19011555,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19012447,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
pivot_value=pivot.values
pivot_value

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [22]:
#pivot.fillna( 0, inplace = True )

In [23]:
pivot.shape

(2375, 3455)

In [24]:
quant= sum_df.QUANTITY.unique().shape[0]
quant

15

# Calculate the similar

mean of quantity

In [25]:
import numpy as np

def mean_of_array(x):
    valid_value = x[~np.isnan(x)]
    return np.mean(valid_value)

In [26]:
mean_of_array(pivot).head()

100085    0.000421
100522    0.000421
100622    0.000842
100683    0.000421
100684    0.001684
dtype: float64

In [27]:
def user_based_mean_centered(pivot_score_table):
    mean_of_each_row = np.apply_along_axis(mean_of_array , axis=1, arr=pivot_score_table)
    user_mean_centered_rating = (pivot_score_table.transpose() - mean_of_each_row).transpose()
    return user_mean_centered_rating

In [28]:
user_based_mean_centered(pivot_value)

array([[-0.00028944, -0.00028944, -0.00028944, ..., -0.00028944,
        -0.00028944, -0.00028944],
       [-0.00086831, -0.00086831, -0.00086831, ..., -0.00086831,
        -0.00086831, -0.00086831],
       [-0.00028944, -0.00028944, -0.00028944, ..., -0.00028944,
        -0.00028944, -0.00028944],
       ...,
       [-0.00028944, -0.00028944, -0.00028944, ..., -0.00028944,
        -0.00028944, -0.00028944],
       [-0.00028944, -0.00028944, -0.00028944, ..., -0.00028944,
        -0.00028944, -0.00028944],
       [-0.00057887, -0.00057887, -0.00057887, ..., -0.00057887,
        -0.00057887, -0.00057887]])

# z-score

In [29]:
import numpy as np

def std_of_array(x):
    valid_value = x[~np.isnan(x)]
    return np.std(valid_value)

In [30]:
def user_based_z_score_normalize(pivot_score_table):
    mean_of_each_column = np.apply_along_axis(mean_of_array , axis=0, arr=pivot_score_table)
    std_of_each_column = np.apply_along_axis(std_of_array , axis=0, arr=pivot_score_table)
    z_score_normalized = (pivot_score_table - mean_of_each_column)/std_of_each_column
    return z_score_normalized

In [31]:
user_based_z_score_normalize(pivot_value)

array([[-0.02052389, -0.02052389, -0.02903128, ..., -0.02052389,
        -0.02903128, -0.02052389],
       [-0.02052389, -0.02052389, -0.02903128, ..., -0.02052389,
        -0.02903128, -0.02052389],
       [-0.02052389, -0.02052389, -0.02903128, ..., -0.02052389,
        -0.02903128, -0.02052389],
       ...,
       [-0.02052389, -0.02052389, -0.02903128, ..., -0.02052389,
        -0.02903128, -0.02052389],
       [-0.02052389, -0.02052389, -0.02903128, ..., -0.02052389,
        -0.02903128, -0.02052389],
       [-0.02052389, -0.02052389, -0.02903128, ..., -0.02052389,
        -0.02903128, -0.02052389]])

In [32]:
table =user_based_z_score_normalize(pivot[:])
table.head()

,100085,100522,100622,100683,100684,100806,100822,100843,100862,100982,...,99087,99222,99322,99582,99702,99782,99885,99887,99923,99946
19000158,-0.020524,-0.020524,-0.029031,-0.020524,-0.033527,-0.020524,-0.0226,-0.020524,-0.020524,-0.029031,...,-0.020524,-0.020524,-0.020524,-0.020524,-0.020524,-0.020524,-0.029031,-0.020524,-0.029031,-0.020524
19000346,-0.020524,-0.020524,-0.029031,-0.020524,-0.033527,-0.020524,-0.0226,-0.020524,-0.020524,-0.029031,...,-0.020524,-0.020524,-0.020524,-0.020524,-0.020524,-0.020524,-0.029031,-0.020524,-0.029031,-0.020524
19000900,-0.020524,-0.020524,-0.029031,-0.020524,-0.033527,-0.020524,-0.0226,-0.020524,-0.020524,-0.029031,...,-0.020524,-0.020524,-0.020524,-0.020524,-0.020524,-0.020524,-0.029031,-0.020524,-0.029031,-0.020524
19005144,-0.020524,-0.020524,-0.029031,-0.020524,-0.033527,-0.020524,-0.0226,-0.020524,-0.020524,-0.029031,...,-0.020524,-0.020524,-0.020524,-0.020524,-0.020524,-0.020524,-0.029031,-0.020524,-0.029031,-0.020524
19008811,-0.020524,-0.020524,-0.029031,-0.020524,-0.033527,-0.020524,-0.0226,-0.020524,-0.020524,-0.029031,...,-0.020524,-0.020524,-0.020524,-0.020524,-0.020524,-0.020524,-0.029031,-0.020524,-0.029031,-0.020524


# pearson

In [34]:
pearson(table)

KeyboardInterrupt: 

In [30]:
sort_recommend = df_corr.sort_values('99885', ascending=False)
sort_recommend2 = sort_recommend[sort_recommend['99885']>0.3]
sort_recommend2['99885']

105764    0.707107
82567     0.707107
101543    0.707107
202062    0.707107
30503     0.707107
95922     0.707107
151471    0.707107
250323    0.500000
25605     0.500000
4122      0.500000
125562    0.500000
253202    0.471405
115362    0.408248
102044    0.408248
121085    0.408248
21408     0.316228
150742    0.316228
Name: 99885, dtype: float64

# most user

In [31]:
most_user_array = np.array(sort_recommend['99885'], dtype=pd.Series)
most_user_array

array([0.7071067811865475, 0.7071067811865475, 0.7071067811865475, ...,
       0.0, 0.0, 0.0], dtype=object)

In [145]:
pivot.shape

(3455, 2375)

# quantity_array

In [33]:
quantity1 = pivot['19023350'].values
quantity1

array([0, 0, 0, ..., 0, 0, 0])

In [34]:
np.unique(quantity1, axis=0)

array([0, 2, 4])

# predict item

In [39]:
array_q = pivot.values
array_q


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [47]:
array_q.shape

(3455, 2375)

In [42]:
for row in array_q:
     sum(row * most_user_array) / sum(most_user_array)
        

ValueError: operands could not be broadcast together with shapes (2375,) (3455,) 

In [43]:
recommend =[]
def predict(quant):
    
    n = len(pivot)
    for i in quant:
        result =  sum(i * most_user_array) / sum(most_user_array)
        return recommend.append(result)
       

In [44]:
predict(quantity1)

In [45]:
recommend

[0.0]

# normalzation

In [166]:
pivot_value = pivot.values
pivot_value.shape

(3455, 2375)

In [167]:
mean = pivot.mean(axis=1).as_matrix()
mean.shape

(3455,)

In [168]:
pivot.std(axis=1).as_matrix()

array([0.02051957, 0.02051957, 0.02901294, ..., 0.0615587 , 0.02901294,
       0.02051957])

In [179]:
def normalization(value):
    mean =pivot.mean(axis=1).as_matrix()
    std = pivot.std(axis=1).as_matrix()
    pivot_value = pivot.values
    
    h = (pivot_value-mean)/std
    r = user_sim*h
  
    
    for corr in user_sim:
        sum_sim = abs(corr)
    
    for i in r:
        rate_nor = mean+(np.sum(i))/(np.sum(sum_sim))
        print (rate_nor)

In [180]:
normalization(pivot_value)

ValueError: operands could not be broadcast together with shapes (3455,2375) (3455,) 